In [ ]:
import os
import wave
import numpy as np
import matplotlib.pyplot as plt
from sighelpers import voice_from_signal, format_sig

if __name__ == '__main__':

    for wav_file in os.listdir('train')[:100]:
        print(wav_file)
        w = wave.open('train/' + wav_file)
        framerate = w.getframerate()
        frames = w.getnframes()
        channels = w.getnchannels()
        width = w.getsampwidth()       
        data = w.readframes(frames)
        sig = format_sig(data, channels)
        tab = voice_from_signal(sig, framerate, low = 20, high = 280, width = 4096)
                
        print(np.mean(tab))
        print("-----")